In [1]:
%matplotlib inline
import pandas as pd
#from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, MaxPooling2D, Dropout, BatchNormalization, Activation, GlobalAveragePooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image
from tensorflow.keras.regularizers import l2
from tensorflow.keras import layers
from tensorflow.keras.callbacks import ReduceLROnPlateau, History, ModelCheckpoint, EarlyStopping
from tensorflow.keras import regularizers
from tensorflow.keras.optimizers import Adam, RMSprop, SGD

train_datagen = ImageDataGenerator(
    rescale=1./255.,
    validation_split=0.25
)

validation_datagen = ImageDataGenerator(
    rescale=1./255.,
    validation_split=0.25
)

train_generator = train_datagen.flow_from_directory(
    "/kaggle/input/drowsiness/eyeImagesSeperated/eyeImagesSeperated/",
    subset="training",
    batch_size=32,
    seed=42,
    shuffle=True,
    class_mode='categorical',
    classes = ['open','closed'],
    target_size=(224, 224)
)

valid_generator = validation_datagen.flow_from_directory(
    "/kaggle/input/drowsiness/eyeImagesSeperated/eyeImagesSeperated/",
    subset="validation",
    batch_size=32,
    seed=42,
    shuffle=True,
    class_mode='categorical',
    classes = ['open','closed'],
    target_size=(224, 224)
)


history = History()

def create_model():
    model = Sequential()

    model.add(Conv2D(32, kernel_size=(3, 3), activation='relu',
              padding="same", input_shape=(224, 224, 3)))
    model.add(Conv2D(32, kernel_size=(3, 3), padding="same", activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Conv2D(64, kernel_size=(3, 3), padding="same",
              activation='relu', kernel_regularizer=regularizers.l2(0.01)))
    model.add(Conv2D(64, kernel_size=(3, 3), padding="same", activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2)))
    # strides=(2,2)
    model.add(Dropout(0.2))

    model.add(Conv2D(128, kernel_size=(3, 3), padding="same",
              activation='relu', kernel_regularizer=regularizers.l2(0.01)))
    model.add(Conv2D(128, kernel_size=(3, 3),
              padding="same", activation='relu'))
    model.add(Conv2D(128, kernel_size=(3, 3),
              padding="same", activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
    model.add(Dropout(0.2))

    model.add(Conv2D(256, kernel_size=(3, 3), padding="same",
              activation='relu', kernel_regularizer=regularizers.l2(0.01)))
    model.add(Conv2D(256, kernel_size=(3, 3),
              padding="same", activation='relu'))
    model.add(Conv2D(256, kernel_size=(3, 3),
              padding="same", activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
    model.add(Dropout(0.3))

    model.add(Flatten())

    model.add(Dense(512, activation='relu',kernel_regularizer=regularizers.l2(0.01)))
    model.add(Dropout(0.3))

    model.add(Dense(512, activation='relu',kernel_regularizer=regularizers.l2(0.01)))
    model.add(Dropout(0.3))

    model.add(Dense(2, activation="sigmoid"))

    return model

def get_lr_metric(optimizer):
    def lr(y_true, y_pred):
        return optimizer._decayed_lr(tf.float32)
    return lr

optimizer = Adam(learning_rate=0.001)
lr_metric = get_lr_metric(optimizer)


checkpoint = ModelCheckpoint('weight_model.h5', verbose=1)

reduce_lr = ReduceLROnPlateau(
    monitor='val_loss', factor=0.1, patience=15, min_lr=0.00001)

callbacks = [history, checkpoint, reduce_lr]

model = create_model()

model.summary()

model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['categorical_accuracy'])

model.fit(
    train_generator, 
    epochs=80,     
    validation_data=valid_generator, 
    callbacks=callbacks,
    verbose=1
)


In [2]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

plt.plot(history.history['categorical_accuracy'])
plt.plot(history.history['val_categorical_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()


In [3]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
test_datagen= ImageDataGenerator(
    rescale=1./255.,
)

test_generator = test_datagen.flow_from_directory(
    "/kaggle/input/drowsiness/eyeImagesSeperatedTesting/eyeImagesSeperatedTesting/",
    batch_size=32,
    seed=42,
    class_mode='categorical',
    classes = ['open','closed'],
    shuffle=False,
    target_size=(224, 224)
)

In [4]:
import numpy
from tensorflow import keras
import sklearn.metrics as metrics
import matplotlib.pyplot as plt
from sklearn.metrics import ConfusionMatrixDisplay

# model = keras.models.load_model('/kaggle/input/drowsymodel/model1.h5')


test_steps_per_epoch = numpy.math.ceil(test_generator.samples / test_generator.batch_size)

predictions = model.predict(test_generator, steps=test_steps_per_epoch)
# Get most likely class
predicted_classes = numpy.argmax(predictions, axis=1)

true_classes = test_generator.classes
class_labels = list(test_generator.class_indices.keys())   

report = metrics.classification_report(true_classes, predicted_classes, target_names=class_labels)

report2 = metrics.confusion_matrix(test_generator.classes, predicted_classes)

disp = ConfusionMatrixDisplay(confusion_matrix=report2, display_labels=class_labels)

disp.plot(cmap=plt.cm.Blues)
plt.show()

print(report)  
# print(report2)

In [5]:
model.save('/kaggle/working/model1.h5') 